In [1]:
import numpy as np
import tensorflow as _tf
from syft.tensor import tensorflow as tf

# Tensorflow Eager Wishlist:

- build something which feels like you're subclassing tensorflow even if you're not
- GPU support
- autograd support

In [2]:
tf.attributes.object_store

{}

In [3]:
EagerTensor = type(_tf.constant([1,2,3]))

In [4]:
EagerTensor.old__init__ = EagerTensor.__init__

In [5]:
def __init__(self, *args, **kwargs):
    print("initializing")
    return EagerTensor.old__init__(self, *args, **kwargs)

EagerTensor.__init__ = __init__

In [6]:
x = _tf.constant([1,2,3])

initializing


In [7]:
x

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [8]:
y = x + x

In [9]:
type(y)

tensorflow.python.framework.ops.EagerTensor

In [10]:
w = _tf.Variable([[100.0]])
with _tf.GradientTape() as tape:
    y = w + w
    print(y)
    loss = y + y

grad = tape.gradient(loss, w)

initializing
tf.Tensor([[200.]], shape=(1, 1), dtype=float32)
initializing
initializing


In [11]:
loss

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[400.]], dtype=float32)>

In [12]:
grad

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[4.]], dtype=float32)>

In [3]:
# w = tf.Variable([[100.0]]).on(tf.MinusIsMultiplyTensor).on(tf.PlusIsMinusTensor)
x_ = np.array([[1, 2], [3, 4]])
x = tf.Variable(x_)#.on(tf.AbstractTensor)

In [4]:
y = x + x

In [5]:
y

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[2, 4],
       [6, 8]])>

In [12]:
tf.attributes.object_store

NameError: name 'tf' is not defined

In [5]:
y = x + x

creating result wrapper around:tf.Tensor(
[[2 4]
 [6 8]], shape=(2, 2), dtype=int64)
Result Id:14
created result wrapper:<syft.tensor.tensorflow.abstract.AbstractTensor object at 0x7fbd28464cc0>


In [28]:
y

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[2, 4],
       [6, 8]])>

In [24]:
_tf.constant([[2,4],[6,8]])

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2, 4],
       [6, 8]], dtype=int32)>

In [25]:
tf.attributes.object_store

{140450957125000: {'end': False,
  'child': <syft.tensor.tensorflow.abstract.AbstractTensor at 0x7fbd49686518>},
 7070133580906406508456322468669: {'end': False},
 1369217622948774742773766512678: {},
 14: {'child': <syft.tensor.tensorflow.abstract.AbstractTensor at 0x7fbd28464cc0>}}

In [ ]:
# class FixedPrecisionTensor(AbstractTorchTensor):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)

#         self.base = 10
#         self.precision_fractional = 3

#         self.data = self.data * self.scaling_factor

#     @property
#     def scaling_factor(self):
#         return self.base ** self.precision_fractional


# method_name = "__add__"

# x_ = np.array([[1, 2], [3, 4]])
# target = x_.__getattribute__(method_name)(x_)

# x = tf.Variable(x_).on(tf.AbstractTensor)
# out = x.__getattribute__(method_name)(x)

# # assert isinstance(target, np.ndarray)
# # assert isinstance(out.child, tf.AbstractTensor)
# # assert (out.numpy() == target).all()

In [16]:
out = w@w

In [17]:
out

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[10000.]], dtype=float32)>

In [15]:
w + w

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[200.]], dtype=float32)>

In [19]:
grad

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[4.]], dtype=float32)>

In [4]:
loss

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[400.]], dtype=float32)>